# Example: Parameter Uncertainty in Single Index Models (SIMs)
In this example, students will explore the question of parameter uncertainty in single index models. Single index models are widely used in finance to model the growth rate (return) of assets based on firm-specific factors and their relationship with a market index.

> __Learning Objectives:__
>
> Fill me in

Let's go!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

## Task 1: Estimate Single Index Models for All Firms in our Dataset
Fill me in.

## Task 2: Compute R-Squared Values for Each Firm's SIM
Fill me in.

## Task 3: Compute Parameter Uncertainty using Bootstrapping
Fill me in.

## Disclaimer and Risks

__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team.

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.